<a href="https://colab.research.google.com/github/Bitachi/parking-detector-poc-ipynb/blob/main/parking_detector_poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#必要なライブラリをインストール
!pip install ultralytics deep_sort_realtime opencv-python-headless --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.9 MB/s eta 0:00:00


In [2]:
#検証対象の動画ファイルおよびYOLOモデルをダウンロード
!wget "https://parking-detection-source.s3.ap-northeast-1.amazonaws.com/fixedPointCamera.mp4" -O /content/fixedPointCameraAkihabara.mp4
!wget "https://parking-detection-source.s3.ap-northeast-1.amazonaws.com/sapporo_2.mp4" -O /content/fixedPointCameraSapporo.mp4
!wget "https://parking-detection-source.s3.ap-northeast-1.amazonaws.com/yolov8l.pt" -O /content/yolov8l.pt
!wget "https://parking-detection-source.s3.ap-northeast-1.amazonaws.com/yolov8m.pt" -O /content/yolov8m.pt
!wget "https://parking-detection-source.s3.ap-northeast-1.amazonaws.com/yolov8n.pt" -O /content/yolov8n.pt
!wget "https://parking-detection-source.s3.ap-northeast-1.amazonaws.com/yolov5s.pt" -O /content/yolov5s.pt

--2025-06-01 12:16:19--  https://parking-detection-source.s3.ap-northeast-1.amazonaws.com/fixedPointCamera.mp4
Resolving parking-detection-source.s3.ap-northeast-1.amazonaws.com (parking-detection-source.s3.ap-northeast-1.amazonaws.com)... 52.219.16.99, 3.5.156.242, 3.5.158.238, ...
Connecting to parking-detection-source.s3.ap-northeast-1.amazonaws.com (parking-detection-source.s3.ap-northeast-1.amazonaws.com)|52.219.16.99|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 463496808 (442M) [video/mp4]
Saving to: ‘/content/fixedPointCameraAkihabara.mp4’

/content/fixedPoint 100%[===================>] 442.02M  20.2MB/s    in 24s     

2025-06-01 12:16:44 (18.4 MB/s) - ‘/content/fixedPointCameraAkihabara.mp4’ saved [463496808/463496808]

--2025-06-01 12:16:44--  https://parking-detection-source.s3.ap-northeast-1.amazonaws.com/sapporo_2.mp4
Resolving parking-detection-source.s3.ap-northeast-1.amazonaws.com (parking-detection-source.s3.ap-northeast-1.amazonaws.com)...

In [2]:
import os

# モデルファイルと動画ファイルの存在確認
model_file_path = "/content/yolov8m.pt"
video_type = "Akihabara"
#video_type = "Sapporo"
video_file_path = "/content/fixedPointCamera" + video_type + ".mp4"

if not os.path.exists(model_file_path):
    print(f"エラー: モデルファイルが見つかりません: {model_file_path}")
    print("'/content/' ディレクトリに 'yolov8m.pt' をアップロードしてください。")
elif not os.path.exists(video_file_path):
    print(f"エラー: 動画ファイルが見つかりません: {video_file_path}")
    print("'/content/' ディレクトリに 'fixedPointCamera.mp4' をアップロードしてください。")
else:
    print("モデルファイルと動画ファイルの準備ができました。")

モデルファイルと動画ファイルの準備ができました。


In [4]:
import cv2
import torch
from ultralytics import YOLO
import pandas as pd
import time
import os
from collections import deque
import numpy as np
from google.colab.patches import cv2_imshow # Colabでの画像表示用
from google.colab import drive # Google Drive連携用
import shutil # ファイル/ディレクトリ操作用

# Google Driveをマウント
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
    # Google Drive上の出力先ベースディレクトリを指定
    # 例: MyDrive直下に "ParkingDetectionOutput" フォルダを作成し、その中に保存
    # このディレクトリは事前に作成しておくか、コード内で作成処理を追加することも可能です。
    DRIVE_OUTPUT_BASE_DIR = '/content/drive/MyDrive/ParkingDetectionOutput'
    if not os.path.exists(DRIVE_OUTPUT_BASE_DIR):
        os.makedirs(DRIVE_OUTPUT_BASE_DIR)
        print(f"Created Google Drive directory: {DRIVE_OUTPUT_BASE_DIR}")
except Exception as e:
    print(f"Failed to mount Google Drive: {e}")
    DRIVE_OUTPUT_BASE_DIR = None # Driveマウント失敗時はNoneに

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.
Created Google Drive directory: /content/drive/MyDrive/ParkingDetectionOutput


In [5]:
# このセルを実行する前に、model_file_path, video_file_path, video_type が定義されている必要があります。
# 例:
model_file_path = "/content/yolov8m.pt"
video_type = "Akihabara"
#video_type = "Sapporo"
video_file_path = "/content/fixedPointCamera" + video_type + ".mp4"

# 0. 設定値
PROCESS_DURATION_SECONDS = 180
FRAMES_PER_SECOND_TO_PROCESS = 1
PARKING_THRESHOLD_SECONDS = 10
CONFIDENCE_THRESHOLD = 0.15
MOVEMENT_THRESHOLD_RATIO = 0.10
TARGET_CLASS_IDS = [2, 5, 7]

MODEL_PATH = model_file_path
VIDEO_PATH = video_file_path

# --- 出力パス設定 (変更・追加) ---
# ローカル（Colab環境）の一時的な出力ディレクトリ
LOCAL_BASE_OUTPUT_DIR = "/content/output_data"
if not os.path.exists(LOCAL_BASE_OUTPUT_DIR):
    os.makedirs(LOCAL_BASE_OUTPUT_DIR)

# JPEG画像出力用サブディレクトリ
LOCAL_JPG_OUTPUT_DIR = os.path.join(LOCAL_BASE_OUTPUT_DIR, "frames_" + video_type + "_" + str(CONFIDENCE_THRESHOLD))
if not os.path.exists(LOCAL_JPG_OUTPUT_DIR):
    os.makedirs(LOCAL_JPG_OUTPUT_DIR)

OUTPUT_VIDEO_PATH = os.path.join(LOCAL_BASE_OUTPUT_DIR, "output_parking_detection_" + video_type + "_" + str(CONFIDENCE_THRESHOLD) +  ".mp4")
CSV_PATH = os.path.join(LOCAL_BASE_OUTPUT_DIR, "parking_log_" + video_type + "_" + str(CONFIDENCE_THRESHOLD) + ".csv")

# Google Drive上の出力先ディレクトリ (実行IDなどでユニークにすると良い)
# ここではvideo_typeとconfidenceでサブフォルダを作成
if DRIVE_OUTPUT_BASE_DIR:
    DRIVE_SPECIFIC_OUTPUT_DIR = os.path.join(DRIVE_OUTPUT_BASE_DIR, f"output_{video_type}_conf{CONFIDENCE_THRESHOLD}_{int(time.time())}")
    if not os.path.exists(DRIVE_SPECIFIC_OUTPUT_DIR):
        os.makedirs(DRIVE_SPECIFIC_OUTPUT_DIR)
    print(f"Google Drive output will be saved to: {DRIVE_SPECIFIC_OUTPUT_DIR}")
else:
    DRIVE_SPECIFIC_OUTPUT_DIR = None
    print("Google Drive not mounted. Output will only be saved locally in Colab.")
# --------------------------------

Google Drive output will be saved to: /content/drive/MyDrive/ParkingDetectionOutput/output_Akihabara_conf0.15_1748781133


In [7]:


def run_parking_detection():
    # 1. モデルのロード
    print(f"Loading model from {MODEL_PATH}...")
    try:
        model = YOLO(MODEL_PATH)
    except Exception as e:
        print(f"Error loading YOLO model: {e}")
        return

    print("Model loaded successfully.")
    if torch.cuda.is_available():
        print("CUDA is available. Using GPU.")
        model.to('cuda')
    else:
        print("CUDA not available. Using CPU.")

    # 2. 動画の読み込みと設定
    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        print(f"Error: Could not open video {VIDEO_PATH}")
        return

    video_fps = cap.get(cv2.CAP_PROP_FPS)
    video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    if video_fps > 0 and FRAMES_PER_SECOND_TO_PROCESS > 0:
        frame_skip_interval = max(1, int(round(video_fps / FRAMES_PER_SECOND_TO_PROCESS)))
    else:
        frame_skip_interval = 1
        if FRAMES_PER_SECOND_TO_PROCESS <=0:
            print(f"Warning: FRAMES_PER_SECOND_TO_PROCESS ({FRAMES_PER_SECOND_TO_PROCESS}) is invalid.")

    print(f"Video Info: FPS={video_fps:.2f}, Size=({video_width}x{video_height}), SkipInterval={frame_skip_interval}")
    print(f"Using CONFIDENCE_THRESHOLD: {CONFIDENCE_THRESHOLD}")
    print(f"Using MOVEMENT_THRESHOLD_RATIO: {MOVEMENT_THRESHOLD_RATIO}")
    print(f"Processed frames will be saved to: {LOCAL_JPG_OUTPUT_DIR}")


    output_fps = FRAMES_PER_SECOND_TO_PROCESS
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_video = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, output_fps, (video_width, video_height))

    # 3. データ構造の初期化
    tracked_vehicles = {}
    parking_log_list = []

    frame_counter = 0
    processed_frame_count = 0
    last_processed_time_sec = 0.0

    print("Starting video processing...")
    start_time_processing = time.time()

    while cap.isOpened():
        ret, frame_original = cap.read() # 描画前のオリジナルフレームを保持
        if not ret:
            print("End of video or error reading frame.")
            break

        current_frame_time_msec = cap.get(cv2.CAP_PROP_POS_MSEC)
        current_frame_time_sec = current_frame_time_msec / 1000.0

        if current_frame_time_sec > PROCESS_DURATION_SECONDS:
            print(f"Reached processing duration limit ({PROCESS_DURATION_SECONDS}s).")
            break

        # フレームをコピーして描画用に使用
        frame_to_draw = frame_original.copy()

        if frame_counter % frame_skip_interval == 0:
            processed_frame_count += 1
            last_processed_time_sec = current_frame_time_sec

            # 4. 検知と追跡 (YOLOv8)
            results = model.track(frame_original, persist=True, classes=TARGET_CLASS_IDS, conf=CONFIDENCE_THRESHOLD, verbose=False, tracker="bytetrack.yaml")

            if results[0].boxes is not None and results[0].boxes.id is not None:
                boxes = results[0].boxes.xyxy.cpu().numpy()
                track_ids = results[0].boxes.id.cpu().numpy().astype(int)
                confs = results[0].boxes.conf.cpu().numpy()

                for i, track_id in enumerate(track_ids):
                    x1, y1, x2, y2 = map(int, boxes[i])
                    w, h = x2 - x1, y2 - y1
                    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2
                    current_conf = confs[i]

                    if track_id not in tracked_vehicles:
                        tracked_vehicles[track_id] = {
                            'positions': deque(maxlen=int(FRAMES_PER_SECOND_TO_PROCESS * (PARKING_THRESHOLD_SECONDS + 5))),
                            'is_parking': False,
                            'parking_start_time': None,
                            'last_seen_time': current_frame_time_sec
                        }

                    vehicle_data = tracked_vehicles[track_id]
                    vehicle_data['positions'].append((current_frame_time_sec, cx, cy, w, h))
                    vehicle_data['last_seen_time'] = current_frame_time_sec

                    # 5. 停車判定ロジック
                    min_data_points_for_parking_decision = int(FRAMES_PER_SECOND_TO_PROCESS * PARKING_THRESHOLD_SECONDS)
                    if len(vehicle_data['positions']) >= min_data_points_for_parking_decision:
                        latest_time_in_deque = vehicle_data['positions'][-1][0]
                        cutoff_time_for_parking_check = latest_time_in_deque - PARKING_THRESHOLD_SECONDS
                        positions_in_window = [p for p in vehicle_data['positions'] if p[0] >= cutoff_time_for_parking_check]

                        if len(positions_in_window) >= min_data_points_for_parking_decision * 0.8:
                            first_pos_data = positions_in_window[0]
                            max_dist_sq = 0
                            sum_w, sum_h = 0,0
                            for pos_data_idx, pos_data in enumerate(positions_in_window):
                                dist_sq = (pos_data[1] - first_pos_data[1])**2 + (pos_data[2] - first_pos_data[2])**2
                                if dist_sq > max_dist_sq: max_dist_sq = dist_sq
                                sum_w += pos_data[3]; sum_h += pos_data[4]
                            max_movement_distance = np.sqrt(max_dist_sq)
                            num_pos_in_window = len(positions_in_window)
                            avg_w = sum_w / num_pos_in_window if num_pos_in_window > 0 else 0
                            avg_h = sum_h / num_pos_in_window if num_pos_in_window > 0 else 0
                            movement_threshold = min(avg_w, avg_h) * MOVEMENT_THRESHOLD_RATIO if min(avg_w, avg_h) > 0 else float('inf')
                            is_currently_still = max_movement_distance < movement_threshold

                            if not vehicle_data['is_parking'] and is_currently_still:
                                vehicle_data['is_parking'] = True
                                vehicle_data['parking_start_time'] = positions_in_window[0][0]
                            elif vehicle_data['is_parking'] and not is_currently_still:
                                vehicle_data['is_parking'] = False
                                parking_end_time = current_frame_time_sec
                                if vehicle_data['parking_start_time'] is not None:
                                    actual_parking_duration = parking_end_time - vehicle_data['parking_start_time']
                                    if actual_parking_duration >= PARKING_THRESHOLD_SECONDS:
                                        parking_log_list.append({'car_id': track_id, 'start_time': round(vehicle_data['parking_start_time'], 2), 'end_time': round(parking_end_time, 2)})
                                vehicle_data['parking_start_time'] = None

                    # 6. 結果の描画 (frame_to_draw に対して行う)
                    color = (0, 255, 0)
                    text = f"ID:{track_id} C:{current_conf:.2f}"
                    if vehicle_data['is_parking']:
                        color = (0, 0, 255)
                        text += f" (P:{vehicle_data['parking_start_time']:.0f}s)"
                    cv2.rectangle(frame_to_draw, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(frame_to_draw, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            cv2.putText(frame_to_draw, f"Time: {current_frame_time_sec:.2f}s", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # MP4動画にフレームを書き込み
            out_video.write(frame_to_draw)

            # JPEG画像としてフレームを保存 (追加)
            # ファイル名は processed_frame_count (1から始まる連番) を使用
            jpg_filename = os.path.join(LOCAL_JPG_OUTPUT_DIR, f"frame_{processed_frame_count:04d}.jpg")
            cv2.imwrite(jpg_filename, frame_to_draw)

        frame_counter += 1
        if frame_counter % (10 * frame_skip_interval) == 0: # 10処理フレームごとに進捗表示
             elapsed_time = time.time() - start_time_processing
             print(f"Processed {processed_frame_count} frames (video time: {current_frame_time_sec:.2f}s). Saved {processed_frame_count} JPGs. Elapsed: {elapsed_time:.2f}s")

    print("Video processing finished.")
    print(f"Total frames processed and saved as JPG: {processed_frame_count}")

    # 7. ループ終了後の処理 (停車ログの最終処理)
    for track_id, data in tracked_vehicles.items():
        if data['is_parking'] and data['parking_start_time'] is not None:
            parking_end_time = last_processed_time_sec
            actual_parking_duration = parking_end_time - data['parking_start_time']
            if actual_parking_duration >= PARKING_THRESHOLD_SECONDS:
                 parking_log_list.append({'car_id': track_id, 'start_time': round(data['parking_start_time'], 2), 'end_time': round(parking_end_time, 2)})

    # 8. CSVファイルへの記録
    if parking_log_list:
        parking_df = pd.DataFrame(parking_log_list)
        parking_df.sort_values(by=['car_id', 'start_time'], inplace=True)
        parking_df.to_csv(CSV_PATH, index=False)
        print(f"Parking log saved locally to {CSV_PATH}")
        # print(parking_df.head())
    else:
        print("No parking events to log.")
        pd.DataFrame(columns=['car_id', 'start_time', 'end_time']).to_csv(CSV_PATH, index=False) # 空のCSVを作成

    # 9. リソース解放
    cap.release()
    out_video.release()
    if 'google.colab' in str(get_ipython()): # Colab環境以外での cv2.destroyAllWindows() エラーを回避
      cv2.destroyAllWindows()
    print(f"Output video saved locally to {OUTPUT_VIDEO_PATH}")
    print("Local cleanup complete.")

    # 10. Google Driveへのファイル保存 (追加)
    if DRIVE_SPECIFIC_OUTPUT_DIR:
        print(f"\nStarting to copy files to Google Drive: {DRIVE_SPECIFIC_OUTPUT_DIR}")
        try:
            # MP4動画のコピー
            if os.path.exists(OUTPUT_VIDEO_PATH):
                shutil.copy(OUTPUT_VIDEO_PATH, os.path.join(DRIVE_SPECIFIC_OUTPUT_DIR, os.path.basename(OUTPUT_VIDEO_PATH)))
                print(f"Copied MP4 to Drive: {os.path.basename(OUTPUT_VIDEO_PATH)}")

            # CSVファイルのコピー
            if os.path.exists(CSV_PATH):
                shutil.copy(CSV_PATH, os.path.join(DRIVE_SPECIFIC_OUTPUT_DIR, os.path.basename(CSV_PATH)))
                print(f"Copied CSV to Drive: {os.path.basename(CSV_PATH)}")

            # JPEG画像群のディレクトリごとコピー
            if os.path.exists(LOCAL_JPG_OUTPUT_DIR) and os.listdir(LOCAL_JPG_OUTPUT_DIR): # ディレクトリが存在し、空でない場合
                drive_jpg_dir_name = os.path.basename(LOCAL_JPG_OUTPUT_DIR)
                shutil.copytree(LOCAL_JPG_OUTPUT_DIR, os.path.join(DRIVE_SPECIFIC_OUTPUT_DIR, drive_jpg_dir_name))
                print(f"Copied JPG frames directory to Drive: {drive_jpg_dir_name}")
            elif not os.listdir(LOCAL_JPG_OUTPUT_DIR):
                 print(f"JPG directory {LOCAL_JPG_OUTPUT_DIR} is empty. Not copying to Drive.")


            print("All files copied to Google Drive successfully.")
        except Exception as e:
            print(f"Error copying files to Google Drive: {e}")
    else:
        print("\nGoogle Drive not configured. Skipping copy to Drive.")
        print("You can download files manually from the Colab environment's /content/output_data/ directory.")

# このセルを実行する前に、model_file_path, video_file_path, video_type が定義されている必要があります。
if __name__ == "__main__" and 'google.colab' in str(get_ipython()):
    if os.path.exists(MODEL_PATH) and os.path.exists(VIDEO_PATH):
        run_parking_detection()
    else:
        print("Please ensure model and video files are uploaded to /content/ and that 'video_type' is defined.")
        print("Example: model_file_path = '/content/yolov8m.pt'; video_file_path = '/content/your_video.mp4'; video_type = 'test_run'")

Loading model from /content/yolov8m.pt...
Model loaded successfully.
CUDA not available. Using CPU.
Video Info: FPS=30.00, Size=(640x360), SkipInterval=30
Using CONFIDENCE_THRESHOLD: 0.15
Using MOVEMENT_THRESHOLD_RATIO: 0.1
Processed frames will be saved to: /content/output_data/frames_Akihabara_0.15
Starting video processing...
Processed 10 frames (video time: 9.97s). Saved 10 JPGs. Elapsed: 10.55s
Processed 20 frames (video time: 19.97s). Saved 20 JPGs. Elapsed: 20.51s
Processed 30 frames (video time: 29.97s). Saved 30 JPGs. Elapsed: 28.89s
Processed 40 frames (video time: 39.97s). Saved 40 JPGs. Elapsed: 38.82s
Processed 50 frames (video time: 49.97s). Saved 50 JPGs. Elapsed: 48.69s
Processed 60 frames (video time: 59.97s). Saved 60 JPGs. Elapsed: 57.12s
Processed 70 frames (video time: 69.97s). Saved 70 JPGs. Elapsed: 67.12s
Processed 80 frames (video time: 79.97s). Saved 80 JPGs. Elapsed: 77.16s
Processed 90 frames (video time: 89.97s). Saved 90 JPGs. Elapsed: 87.59s
Processed 100

In [8]:

#旧ver
import cv2
import torch
from ultralytics import YOLO
import pandas as pd
import time
import os
from collections import deque
import numpy as np
from google.colab.patches import cv2_imshow # Colabでの画像表示用

#YOLOのConfidenceを設定できるように改良

# 0. 設定値
PROCESS_DURATION_SECONDS = 180  # 動画の処理対象時間（秒）
FRAMES_PER_SECOND_TO_PROCESS = 1  # 1秒あたりに処理するフレーム数
PARKING_THRESHOLD_SECONDS = 10  # 停車とみなす時間（秒）

# --- フィードバック対応パラメータ ---
# YOLO検出時のConfidence閾値 (0.0 から 1.0 の間。低いほど検出しやすいが誤検出も増える)
# デフォルトはYOLOライブラリ側で0.25程度。少し下げてみます。
CONFIDENCE_THRESHOLD = 0.15 # 例: 15%

# 位置変動の閾値（バウンディングボックスの短辺の割合）
# この値を大きくすると、多少の揺れや微動があっても停車と判定されやすくなる
# デフォルト: 0.05 (5%) -> 少し緩めてみます
MOVEMENT_THRESHOLD_RATIO = 0.10 # 例: 10%
# ---------------------------------

MODEL_PATH = model_file_path
VIDEO_PATH = video_file_path
OUTPUT_VIDEO_PATH = "/content/output_parking_detection_" + video_type + "_" + str(CONFIDENCE_THRESHOLD) +  ".mp4"
CSV_PATH = "/content/parking_log_" + video_type + "_" + str(CONFIDENCE_THRESHOLD) + ".csv"

TARGET_CLASS_IDS = [2, 5, 7] # 検知対象クラスID: 2 (car), 5 (bus), 7 (truck)

def run_parking_detection():
    # 1. モデルのロード
    print(f"Loading model from {MODEL_PATH}...")
    try:
        model = YOLO(MODEL_PATH)
    except Exception as e:
        print(f"Error loading YOLO model: {e}")
        return

    print("Model loaded successfully.")
    if torch.cuda.is_available():
        print("CUDA is available. Using GPU.")
        model.to('cuda')
    else:
        print("CUDA not available. Using CPU.")

    # 2. 動画の読み込みと設定
    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        print(f"Error: Could not open video {VIDEO_PATH}")
        return

    video_fps = cap.get(cv2.CAP_PROP_FPS)
    video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    if video_fps > 0 and FRAMES_PER_SECOND_TO_PROCESS > 0:
        frame_skip_interval = max(1, int(round(video_fps / FRAMES_PER_SECOND_TO_PROCESS)))
    else:
        frame_skip_interval = 1
        if FRAMES_PER_SECOND_TO_PROCESS <=0:
            print(f"Warning: FRAMES_PER_SECOND_TO_PROCESS ({FRAMES_PER_SECOND_TO_PROCESS}) is invalid. Defaulting to processing all frames selected by skip interval.")

    print(f"Video Info: FPS={video_fps:.2f}, Size=({video_width}x{video_height}), SkipInterval={frame_skip_interval}")
    print(f"Using CONFIDENCE_THRESHOLD: {CONFIDENCE_THRESHOLD}")
    print(f"Using MOVEMENT_THRESHOLD_RATIO: {MOVEMENT_THRESHOLD_RATIO}")

    output_fps = FRAMES_PER_SECOND_TO_PROCESS
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_video = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, output_fps, (video_width, video_height))

    # 3. データ構造の初期化
    tracked_vehicles = {}
    parking_log_list = []

    frame_counter = 0
    processed_frame_count = 0
    last_processed_time_sec = 0.0

    print("Starting video processing...")
    start_time_processing = time.time()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video or error reading frame.")
            break

        current_frame_time_msec = cap.get(cv2.CAP_PROP_POS_MSEC)
        current_frame_time_sec = current_frame_time_msec / 1000.0

        if current_frame_time_sec > PROCESS_DURATION_SECONDS:
            print(f"Reached processing duration limit ({PROCESS_DURATION_SECONDS}s).")
            break

        if frame_counter % frame_skip_interval == 0:
            processed_frame_count += 1
            last_processed_time_sec = current_frame_time_sec

            # 4. 検知と追跡 (YOLOv8)
            results = model.track(frame, persist=True, classes=TARGET_CLASS_IDS, conf=CONFIDENCE_THRESHOLD, verbose=False, tracker="bytetrack.yaml")

            if results[0].boxes is not None and results[0].boxes.id is not None:
                boxes = results[0].boxes.xyxy.cpu().numpy()
                track_ids = results[0].boxes.id.cpu().numpy().astype(int)
                confs = results[0].boxes.conf.cpu().numpy() # Confidenceの値を取得

                for i, track_id in enumerate(track_ids): # enumerateを使用してインデックスiを取得
                    x1, y1, x2, y2 = map(int, boxes[i])
                    w, h = x2 - x1, y2 - y1
                    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2
                    current_conf = confs[i] # 現在の検出のConfidence

                    if track_id not in tracked_vehicles:
                        tracked_vehicles[track_id] = {
                            'positions': deque(maxlen=int(FRAMES_PER_SECOND_TO_PROCESS * (PARKING_THRESHOLD_SECONDS + 5))),
                            'is_parking': False,
                            'parking_start_time': None,
                            'last_seen_time': current_frame_time_sec
                        }

                    vehicle_data = tracked_vehicles[track_id]
                    vehicle_data['positions'].append((current_frame_time_sec, cx, cy, w, h))
                    vehicle_data['last_seen_time'] = current_frame_time_sec

                    # 5. 停車判定ロジック (変更なし)
                    min_data_points_for_parking_decision = int(FRAMES_PER_SECOND_TO_PROCESS * PARKING_THRESHOLD_SECONDS)
                    if len(vehicle_data['positions']) >= min_data_points_for_parking_decision:
                        latest_time_in_deque = vehicle_data['positions'][-1][0]
                        cutoff_time_for_parking_check = latest_time_in_deque - PARKING_THRESHOLD_SECONDS
                        positions_in_window = [p for p in vehicle_data['positions'] if p[0] >= cutoff_time_for_parking_check]

                        if len(positions_in_window) >= min_data_points_for_parking_decision * 0.8:
                            first_pos_data = positions_in_window[0]
                            max_dist_sq = 0
                            sum_w, sum_h = 0,0
                            for pos_data_idx, pos_data in enumerate(positions_in_window):
                                dist_sq = (pos_data[1] - first_pos_data[1])**2 + (pos_data[2] - first_pos_data[2])**2
                                if dist_sq > max_dist_sq: max_dist_sq = dist_sq
                                sum_w += pos_data[3]; sum_h += pos_data[4]
                            max_movement_distance = np.sqrt(max_dist_sq)
                            num_pos_in_window = len(positions_in_window)
                            avg_w = sum_w / num_pos_in_window if num_pos_in_window > 0 else 0
                            avg_h = sum_h / num_pos_in_window if num_pos_in_window > 0 else 0
                            movement_threshold = min(avg_w, avg_h) * MOVEMENT_THRESHOLD_RATIO if min(avg_w, avg_h) > 0 else float('inf')
                            is_currently_still = max_movement_distance < movement_threshold

                            if not vehicle_data['is_parking'] and is_currently_still:
                                vehicle_data['is_parking'] = True
                                vehicle_data['parking_start_time'] = positions_in_window[0][0]
                            elif vehicle_data['is_parking'] and not is_currently_still:
                                vehicle_data['is_parking'] = False
                                parking_end_time = current_frame_time_sec
                                if vehicle_data['parking_start_time'] is not None:
                                    actual_parking_duration = parking_end_time - vehicle_data['parking_start_time']
                                    if actual_parking_duration >= PARKING_THRESHOLD_SECONDS:
                                        parking_log_list.append({'car_id': track_id, 'start_time': round(vehicle_data['parking_start_time'], 2), 'end_time': round(parking_end_time, 2)})
                                vehicle_data['parking_start_time'] = None

                    # 6. 結果の描画 (Confidence表示を追加)
                    color = (0, 255, 0) # 緑: 通常
                    # IDとConfidenceをラベルに表示
                    text = f"ID:{track_id} C:{current_conf:.2f}"
                    if vehicle_data['is_parking']:
                        color = (0, 0, 255) # 赤: 停車中
                        # 停車情報を短縮して表示 (P: Parking start time)
                        text += f" (P:{vehicle_data['parking_start_time']:.0f}s)"

                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2) # フォントサイズを少し調整(0.6->0.5)

            cv2.putText(frame, f"Time: {current_frame_time_sec:.2f}s", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            out_video.write(frame)

        frame_counter += 1
        if frame_counter % (10 * frame_skip_interval) == 0:
             elapsed_time = time.time() - start_time_processing
             print(f"Processed {processed_frame_count} frames. Video time: {current_frame_time_sec:.2f}s. Elapsed: {elapsed_time:.2f}s")

    print("Video processing finished.")
    print(f"Total frames processed: {processed_frame_count}")

    for track_id, data in tracked_vehicles.items():
        if data['is_parking'] and data['parking_start_time'] is not None:
            parking_end_time = last_processed_time_sec
            actual_parking_duration = parking_end_time - data['parking_start_time']
            if actual_parking_duration >= PARKING_THRESHOLD_SECONDS:
                 parking_log_list.append({'car_id': track_id, 'start_time': round(data['parking_start_time'], 2), 'end_time': round(parking_end_time, 2)})

    if parking_log_list:
        parking_df = pd.DataFrame(parking_log_list)
        parking_df.sort_values(by=['car_id', 'start_time'], inplace=True)
        parking_df.to_csv(CSV_PATH, index=False)
        print(f"Parking log saved to {CSV_PATH}")
        print(parking_df.head())
    else:
        print("No parking events to log.")
        pd.DataFrame(columns=['car_id', 'start_time', 'end_time']).to_csv(CSV_PATH, index=False)

    cap.release()
    out_video.release()
    cv2.destroyAllWindows()
    print(f"Output video saved to {OUTPUT_VIDEO_PATH}")
    print("Cleanup complete.")

if __name__ == "__main__" and 'google.colab' in str(get_ipython()):
    if os.path.exists(MODEL_PATH) and os.path.exists(VIDEO_PATH):
        run_parking_detection()
    else:
        print("Please ensure model and video files are uploaded to /content/ and run Cell 2 again.")

Loading model from /content/yolov8m.pt...
Model loaded successfully.
CUDA not available. Using CPU.
Video Info: FPS=30.00, Size=(640x360), SkipInterval=30
Using CONFIDENCE_THRESHOLD: 0.15
Using MOVEMENT_THRESHOLD_RATIO: 0.1
Starting video processing...


KeyboardInterrupt: 